# 🎵 Pipeline de Restauração de Áudio - V3.0 FIXED

## ✅ O QUE FOI CORRIGIDO:

### 🔧 Correções Principais:
1. **Demucs CORRIGIDO** - Agora funciona com GPU, detecta modelo correto, e tem fallback robusto
2. **Noise Reduction MELHORADO** - Muito mais suave, preserva qualidade, não remove partes importantes
3. **Configurações CONSERVADORAS** - Novas configs que preservam qualidade máxima
4. **Processing Chain SIMPLIFICADO** - Menos processamento = melhor qualidade

### ⚠️ IMPORTANTE - LEIA ANTES DE USAR:

**Este pipeline pode DEGRADAR a qualidade se mal configurado!**

**Regra de Ouro:**
- ✅ **SEMPRE comece com CONFIG_CONSERVATIVE ou CONFIG_ULTRA_SAFE**
- ✅ Teste com 1 arquivo primeiro
- ✅ Compare ANTES vs DEPOIS sempre
- ❌ NUNCA use configurações agressivas sem necessidade
- ❌ NUNCA processe tudo em batch sem testar

---

### 🎯 Qual configuração usar?

| Situação | Configuração | Quando usar |
|----------|-------------|-------------|
| Áudio já está bom | `CONFIG_ULTRA_SAFE` | Só precisa ajustar volume |
| Áudio tem qualidade OK | `CONFIG_CONSERVATIVE` | **RECOMENDADO - Use este por padrão** |
| Quer máxima qualidade + GPU | `CONFIG_DEMUCS_QUALITY` | Tem GPU e pode esperar 10-20min |
| Áudio muito degradado | Auto-Configure | Deixa a IA decidir |
| Apenas masterizar | `CONFIG_MASTERING_ONLY` | Áudio já foi limpo |

---

## 📦 1. Instalação

Execute uma vez no início.

In [ ]:
%%capture

# Bibliotecas principais
!pip install -q librosa soundfile scipy matplotlib numpy noisereduce ipywidgets

# Demucs (OPCIONAL - só se quiser usar separação de stems com GPU)
# Descomente a linha abaixo para instalar:
# !pip install -U demucs

print("✓ Instalação completa!")

## 💾 2. Setup: Drive + GPU

In [ ]:
from google.colab import drive
import torch

# Montar Drive
drive.mount('/content/drive')

# Definir caminhos
AUDIO_INPUT_DIR = '/content/drive/MyDrive/00-restore'
AUDIO_OUTPUT_DIR = '/content/drive/MyDrive/00-restore/restored_output'

print(f"✓ Drive montado")
print(f"📁 Entrada: {AUDIO_INPUT_DIR}")
print(f"📁 Saída: {AUDIO_OUTPUT_DIR}")

# Verificar GPU
print("\n" + "="*70)
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"⚡ GPU: {gpu_name} ({vram:.1f} GB VRAM)")
    print("   ✓ Demucs será rápido (~5-10min por arquivo)")
else:
    print("⚠️ SEM GPU")
    print("   Para ativar: Runtime → Change runtime type → GPU (T4)")
    print("   ⚠️ Demucs será MUITO lento sem GPU (30min-2h por arquivo)")
    print("   💡 Recomendo NÃO usar Demucs sem GPU")
print("="*70)

## 📥 3. Baixar Pipeline (Versão Corrigida)

In [ ]:
import os

# Remover versão antiga
!rm -rf /content/audio-pipeline-repo

# Clonar branch com correções
!git clone -b claude/audio-restoration-pipeline-gAFxk \
  https://github.com/guitorte/musicas.git \
  /content/audio-pipeline-repo

print("\n" + "="*70)
print("✓ Pipeline V3 FIXED baixado!")
print("="*70)
print("\nCorreções incluídas:")
print("  ✓ Demucs corrigido (GPU, fallback robusto)")
print("  ✓ Noise reduction melhorado (mais suave)")
print("  ✓ Configurações conservadoras")
print("  ✓ Pipeline otimizado para qualidade")

## 🔧 4. Importar Módulos

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

# Adicionar ao path
sys.path.insert(0, '/content/audio-pipeline-repo/audio-restoration-pipeline')

# Importar módulos
from modules import AudioRestorationPipeline, SpectralAnalyzer
from modules import AdvancedAudioProcessor, auto_configure, create_quick_config

# Importar configurações SEGURAS
sys.path.insert(0, '/content/audio-pipeline-repo/audio-restoration-pipeline')
from SAFE_CONFIGS import (
    CONFIG_ULTRA_SAFE,
    CONFIG_CONSERVATIVE,
    CONFIG_DEMUCS_QUALITY,
    CONFIG_MASTERING_ONLY,
    USAGE_GUIDE
)

# Utilitários
import glob
import json
from pathlib import Path
from IPython.display import Audio, Image, display, HTML

print("✓ Módulos importados!")
print("\n📚 Configurações seguras disponíveis:")
print("  • CONFIG_ULTRA_SAFE - Mínimo processamento")
print("  • CONFIG_CONSERVATIVE - Recomendado (padrão)")
print("  • CONFIG_DEMUCS_QUALITY - Máxima qualidade c/ GPU")
print("  • CONFIG_MASTERING_ONLY - Apenas masterização")
print("\n💡 Digite: print(USAGE_GUIDE) para ver guia completo")

## 📂 5. Listar Arquivos e Selecionar

Execute esta célula para ver todos os arquivos na pasta `00-restore` e escolher qual arquivo deseja restaurar usando o dropdown.

In [ ]:
# Buscar arquivos
audio_files = []
for ext in ['*.mp3', '*.wav', '*.MP3', '*.WAV', '*.m4a', '*.M4A']:
    audio_files.extend(glob.glob(os.path.join(AUDIO_INPUT_DIR, ext)))

audio_files = sorted(set(audio_files))

print(f"✓ Encontrados {len(audio_files)} arquivos\n")
print("="*70)

for i, file in enumerate(audio_files, 1):
    size_mb = os.path.getsize(file) / (1024 * 1024)
    name = Path(file).name
    print(f"{i:2d}. {name[:50]:50s} ({size_mb:6.2f} MB)")

print("="*70)

if not audio_files:
    print(f"\n⚠️ NENHUM arquivo em {AUDIO_INPUT_DIR}")
    print("Verifique o caminho!")
else:
    # Criar widget de seleção
    import ipywidgets as widgets
    from IPython.display import display
    
    # Criar opções com nome do arquivo e tamanho
    file_options = []
    for i, file in enumerate(audio_files):
        size_mb = os.path.getsize(file) / (1024 * 1024)
        name = Path(file).name
        file_options.append((f"{i+1}. {name} ({size_mb:.2f} MB)", i))
    
    # Dropdown para seleção
    file_selector = widgets.Dropdown(
        options=file_options,
        value=0,
        description='Arquivo:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
    
    # Função para mostrar informações do arquivo selecionado
    def on_file_change(change):
        idx = change['new']
        selected_file = audio_files[idx]
        size_mb = os.path.getsize(selected_file) / (1024 * 1024)
        duration_info = ""
        try:
            import librosa
            y, sr = librosa.load(selected_file, sr=None, duration=10)
            total_duration = librosa.get_duration(path=selected_file)
            duration_info = f" | Duração: {total_duration:.1f}s"
        except:
            pass
        info_label.value = f"<b>Selecionado:</b> {Path(selected_file).name}<br><b>Tamanho:</b> {size_mb:.2f} MB{duration_info}"
    
    # Label para mostrar informações
    info_label = widgets.HTML(value="")
    
    # Conectar evento
    file_selector.observe(on_file_change, names='value')
    
    # Disparar evento inicial
    on_file_change({'new': 0})
    
    print("\n🎵 SELECIONE O ARQUIVO PARA RESTAURAR:")
    display(file_selector)
    display(info_label)
    
    # Variável global para uso em outras células
    SELECTED_FILE_INDEX = 0
    
    def get_selected_file():
        return audio_files[file_selector.value]
    
    print("\n💡 Use get_selected_file() para obter o arquivo selecionado")

## 🚀 6. Inicializar Pipeline

In [ ]:
# Criar output dir
os.makedirs(AUDIO_OUTPUT_DIR, exist_ok=True)

# Inicializar
pipeline = AudioRestorationPipeline(
    sr=44100,
    output_base_dir=AUDIO_OUTPUT_DIR,
    log_dir=os.path.join(AUDIO_OUTPUT_DIR, 'logs')
)

print("✓ Pipeline pronto!")
print(f"  Sample Rate: 44100 Hz")
print(f"  Output: {AUDIO_OUTPUT_DIR}")

---

# 🎯 CONFIGURAÇÃO

**ESCOLHA UMA das opções abaixo**

---

## ⭐ OPÇÃO 1: CONSERVADORA (RECOMENDADA)

**Use esta por padrão!** Preserva qualidade máxima.

In [ ]:
# ════════════════════════════════════════════════════════════
# CONFIGURAÇÃO CONSERVADORA - PRESERVA QUALIDADE
# ════════════════════════════════════════════════════════════

CONFIG = CONFIG_CONSERVATIVE.copy()

print("✓ CONFIG_CONSERVATIVE selecionada")
print("\nCaracterísticas:")
print("  • Noise reduction MUITO suave (0.3)")
print("  • Restauração de frequências conservadora")
print("  • SEM separação de stems (rápido)")
print("  • EQ muito leve (+0.5 presence, +0.3 treble)")
print("  • LUFS -14.0 (streaming)")
print("\n💡 Esta config é segura para a maioria dos áudios")
print("\nConfig completa:")
print(json.dumps(CONFIG, indent=2))

## 🛡️ OPÇÃO 2: ULTRA SAFE (Mínimo Processamento)

Use se o áudio já está bom e só precisa ajustar volume.

In [ ]:
# ════════════════════════════════════════════════════════════
# ULTRA SAFE - APENAS NORMALIZAÇÃO
# ════════════════════════════════════════════════════════════

CONFIG = CONFIG_ULTRA_SAFE.copy()

print("✓ CONFIG_ULTRA_SAFE selecionada")
print("\nCaracterísticas:")
print("  • SEM noise reduction")
print("  • SEM restauração de frequências")
print("  • SEM EQ (0.0 em todas as bandas)")
print("  • Apenas remove clicks e normaliza para -14 LUFS")
print("\n💡 Use quando o áudio já está perfeito")
print("\nConfig completa:")
print(json.dumps(CONFIG, indent=2))

## 🔥 OPÇÃO 3: DEMUCS QUALITY (Máxima Qualidade)

⚠️ **Requer GPU! Demora 10-20 minutos por arquivo!**

In [ ]:
# ════════════════════════════════════════════════════════════
# DEMUCS QUALITY - MÁXIMA QUALIDADE COM SEPARAÇÃO DE STEMS
# ════════════════════════════════════════════════════════════

import torch

if not torch.cuda.is_available():
    print("⚠️ ATENÇÃO: GPU NÃO DISPONÍVEL!")
    print("   Demucs sem GPU é MUITO lento (30min-2h por arquivo)")
    print("\n   Recomendo usar CONFIG_CONSERVATIVE ao invés")
    print("\n   Para continuar mesmo assim, mude USE_DEMUCS_WITHOUT_GPU = True")
    USE_DEMUCS_WITHOUT_GPU = False  # Mude para True se quiser forçar
else:
    USE_DEMUCS_WITHOUT_GPU = True
    print(f"✓ GPU detectada: {torch.cuda.get_device_name(0)}")

if USE_DEMUCS_WITHOUT_GPU:
    CONFIG = CONFIG_DEMUCS_QUALITY.copy()
    
    print("\n✓ CONFIG_DEMUCS_QUALITY selecionada")
    print("\nCaracterísticas:")
    print("  • Separa stems com Demucs (4 stems: vocals, drums, bass, other)")
    print("  • Processa cada stem individualmente")
    print("  • Noise reduction moderado (0.4)")
    print("  • Restauração de frequências")
    print("  • EQ balanceado")
    print("\n⏳ TEMPO ESTIMADO:")
    if torch.cuda.is_available():
        print("   ~10-15 minutos por arquivo (com GPU)")
    else:
        print("   ~30min-2h por arquivo (SEM GPU)")
    
    print("\nConfig completa:")
    print(json.dumps(CONFIG, indent=2))
else:
    CONFIG = CONFIG_CONSERVATIVE.copy()
    print("\n⚠️ Usando CONFIG_CONSERVATIVE por segurança")

## 🤖 OPÇÃO 4: AUTO-CONFIGURAÇÃO (IA Decide)

Analisa o áudio e cria configuração automaticamente.

In [ ]:
# ════════════════════════════════════════════════════════════
# AUTO-CONFIGURAÇÃO BASEADA EM IA
# ════════════════════════════════════════════════════════════

if not audio_files:
    print("⚠️ Nenhum arquivo para analisar")
elif 'file_selector' not in dir():
    print("⚠️ Execute a célula '5. Listar Arquivos' primeiro para selecionar um arquivo")
else:
    # Usar arquivo selecionado no dropdown
    test_file = audio_files[file_selector.value]
    
    print(f"🔍 Analisando: {Path(test_file).name}")
    print("="*70)
    
    # Analisar
    analyzer = SpectralAnalyzer()
    analysis = analyzer.analyze_audio(test_file)
    
    # Gerar configuração inteligente
    CONFIG = auto_configure(analysis, verbose=True)
    
    print("\n✓ Configuração inteligente gerada pela IA!")
    print("\nConfig final:")
    print(json.dumps(CONFIG, indent=2))

---

# ▶️ PROCESSAMENTO

---

## 🎯 7. PROCESSAR ARQUIVO SELECIONADO

**Processa o arquivo que você selecionou no dropdown da célula 5.**

**SEMPRE teste com 1 arquivo primeiro!**

In [ ]:
# ════════════════════════════════════════════════════════════
# PROCESSAR ARQUIVO SELECIONADO
# ════════════════════════════════════════════════════════════

if not audio_files:
    print("⚠️ Nenhum arquivo para processar")
elif 'file_selector' not in dir():
    print("⚠️ Execute a célula '5. Listar Arquivos' primeiro para selecionar um arquivo")
else:
    # Usar arquivo selecionado no dropdown
    file_index = file_selector.value
    test_file = audio_files[file_index]
    
    print(f"🎵 Processando: {Path(test_file).name}")
    print(f"   Arquivo {file_index + 1} de {len(audio_files)}")
    print("="*70)
    
    # Avisos
    if CONFIG.get('separate_stems') and CONFIG.get('stem_separation_model') == 'demucs':
        print("\n⏳ DEMUCS ATIVADO")
        if torch.cuda.is_available():
            print("   GPU disponível: ~10-15 minutos")
        else:
            print("   ⚠️ SEM GPU: ~30min-2h (MUITO LENTO!)")
        print("="*70 + "\n")
    
    # PROCESSAR
    result = pipeline.process_audio(
        test_file,
        config=CONFIG
    )
    
    # Resultado
    print("\n" + "="*70)
    print("✓✓✓ PROCESSAMENTO COMPLETO! ✓✓✓")
    print("="*70)
    
    print(f"\n📁 Pasta: {result['output_dir']}")
    print(f"\n🎵 Arquivo final:")
    print(f"   {Path(result['stages']['mastering']['output']).name}")
    
    # Stems
    if 'stem_separation' in result['stages']:
        stems = result['stages']['stem_separation']
        print(f"\n🎸 Stems ({len(stems)}):")
        for stem_name in stems:
            print(f"   • {stem_name}")
    
    # Guardar para próximas células
    LAST_RESULT = result
    LAST_FILE = test_file
    
    print("\n💡 Execute a próxima célula para comparar ANTES vs DEPOIS")

## 📊 8. COMPARAR: ANTES vs DEPOIS

**IMPORTANTE: Sempre compare para verificar se melhorou!**

In [ ]:
# ════════════════════════════════════════════════════════════
# COMPARAÇÃO E ANÁLISE
# ════════════════════════════════════════════════════════════

if 'LAST_RESULT' not in globals():
    print("⚠️ Processe um arquivo primeiro (célula anterior)")
else:
    result = LAST_RESULT
    
    # Visualização
    viz_path = os.path.join(result['output_dir'], 'analysis_visualization.png')
    if os.path.exists(viz_path):
        print("📊 ANÁLISE ESPECTRAL")
        print("="*70)
        display(Image(filename=viz_path))
    
    # Métricas
    analysis_path = os.path.join(result['output_dir'], 'analysis.json')
    if os.path.exists(analysis_path):
        with open(analysis_path, 'r') as f:
            analysis = json.load(f)
        
        print("\n📈 MÉTRICAS DO ORIGINAL")
        print("="*70)
        print(f"Duração: {analysis['duration']:.1f}s")
        print(f"SNR: {analysis['noise_profile']['snr_db']:.1f} dB")
        print(f"LUFS: {analysis['dynamic_range']['lufs_estimate']:.1f}")
        print(f"Peak: {analysis['dynamic_range']['peak_amplitude']:.3f}")
        
        if analysis['clipping_detection']['has_clipping']:
            print(f"⚠️ Clipping: {analysis['clipping_detection']['clip_percentage']:.2f}%")
        
        if analysis['frequency_analysis']['high_freq_loss']:
            cutoff = analysis['frequency_analysis']['high_freq_cutoff']
            print(f"⚠️ Corte de freq em: {cutoff:.0f} Hz")
    
    # COMPARAÇÃO AUDIO
    print("\n\n🎧 COMPARAÇÃO")
    print("="*70)
    print("\n🔴 ORIGINAL (ANTES):")
    display(Audio(LAST_FILE))
    
    print("\n🟢 PROCESSADO (DEPOIS):")
    display(Audio(result['stages']['mastering']['output']))
    
    print("\n⚠️ IMPORTANTE: Ouça os dois e compare!")
    print("   Se o processado soar PIOR, use uma config mais conservadora")
    
    # Stems
    if 'stem_separation' in result['stages']:
        print("\n\n🎸 STEMS SEPARADOS")
        print("="*70)
        
        stems = result['stages']['stem_separation']
        
        if 'vocals' in stems:
            print("\n🎤 VOCAIS:")
            display(Audio(stems['vocals']))
        
        if 'drums' in stems:
            print("\n🥁 BATERIA:")
            display(Audio(stems['drums']))
        
        if 'bass' in stems:
            print("\n🎸 BAIXO:")
            display(Audio(stems['bass']))
        
        if 'other' in stems:
            print("\n🎹 OUTROS:")
            display(Audio(stems['other']))

## 💾 9. Download do Resultado

Baixe o arquivo processado.

In [ ]:
# ════════════════════════════════════════════════════════════
# DOWNLOAD DO ARQUIVO PROCESSADO
# ════════════════════════════════════════════════════════════

if 'LAST_RESULT' in globals():
    from google.colab import files
    
    final_file = LAST_RESULT['stages']['mastering']['output']
    
    print(f"📥 Baixando: {Path(final_file).name}")
    print(f"   Tamanho: {os.path.getsize(final_file) / (1024*1024):.2f} MB")
    
    # Descomente a linha abaixo para baixar:
    # files.download(final_file)
    
    print("\n💡 Descomente a linha acima para fazer download")
else:
    print("⚠️ Processe um arquivo primeiro")

---

# 📚 DOCUMENTAÇÃO

---

## 💡 Guia de Uso Completo

In [ ]:
print(USAGE_GUIDE)

## 🔧 Verificar Instalações

In [ ]:
import subprocess
import torch

print("🔍 Status do Sistema\n")
print("="*70)

# GPU
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("✗ GPU: Não disponível")

# Demucs
try:
    result = subprocess.run(['demucs', '--version'], capture_output=True, text=True, timeout=5)
    print("✓ Demucs: Instalado")
except:
    print("✗ Demucs: Não instalado")
    print("  Para instalar: !pip install -U demucs")

# Arquivos
print(f"\n✓ Arquivos encontrados: {len(audio_files)}")

print("="*70)
print("\n✓ Verificação completa")

---

## 📋 RESUMO

### ✅ Correções Implementadas:

1. **Demucs Corrigido:**
   - Detecta GPU automaticamente
   - Usa modelo htdemucs_ft (melhor qualidade)
   - Fallback robusto para método básico
   - Busca correta da pasta de output

2. **Noise Reduction Melhorado:**
   - Muito mais suave (sigma 2.5)
   - Nunca remove completamente (min 10%)
   - Wet/dry mix preserva original
   - Usa mediana ao invés de média (mais robusto)

3. **Configurações Seguras:**
   - CONFIG_CONSERVATIVE (recomendada)
   - CONFIG_ULTRA_SAFE (mínimo)
   - CONFIG_DEMUCS_QUALITY (máxima)
   - CONFIG_MASTERING_ONLY (só EQ)

### 🎯 Como Usar:

1. Execute células 1-6 (setup)
2. Escolha UMA configuração (células 7-10)
3. Processe 1 arquivo (célula 11)
4. Compare ANTES vs DEPOIS (célula 12)
5. Se ficou bom, continue. Se não, use config mais conservadora.

### ⚠️ LEMBRE-SE:

- **SEMPRE teste com 1 arquivo primeiro**
- **SEMPRE compare antes vs depois**
- **Comece com CONFIG_CONSERVATIVE**
- **Menos processamento = melhor qualidade**

---

**Pipeline V3 FIXED - Audio Restoration**

🎵 Desenvolvido para preservar qualidade máxima 🎵

---